# Challenge C: permeation through a EUROFER pipe

In [1]:
import festim as F
import h_transport_materials as htm
import numpy as np

In [2]:
results_folder = "challenge_B/"

my_model = F.Simulation()

# define mesh
size = 5e-03
vertices = np.linspace(0, size, num=1000)

my_model.mesh = F.MeshFromVertices(vertices=vertices, type="cylindrical")

# define materials

# eurofer properties
D_eurofer = htm.diffusivities.filter(material="eurofer_97").filter(author="chen")[0]

S_eurofer = htm.solubilities.filter(material="eurofer_97").filter(author="chen")[0]

eurofer = F.Material(
    id=1,
    D_0=D_eurofer.pre_exp.magnitude,
    E_D=D_eurofer.act_energy.magnitude,
)
my_model.materials = eurofer

# define temperature
my_model.T = F.Temperature(value=600)

# define boundary conditions
my_model.boundary_conditions = [
    F.SievertsBC(
        surfaces=1,
        S_0=S_eurofer.pre_exp.magnitude,
        E_S=S_eurofer.act_energy.magnitude,
        pressure=1e6,
    ),
    F.DirichletBC(surfaces=2, value=0, field="solute"),
]

# define settings
my_model.settings = F.Settings(
    absolute_tolerance=1e-10,
    relative_tolerance=1e-10,
    transient=False,
)

# define exports
my_derived_quantities = F.DerivedQuantities(
    [F.SurfaceFluxCylindrical("solute", surface=2)],
    show_units=True,
)
my_model.exports = F.Exports([my_derived_quantities])

# run simulation
my_model.initialise()
my_model.run()

Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...
Solved problem in 0.00 s


In [3]:
surface_flux = np.abs(my_derived_quantities[0].data[0])

print(f"Surface flux = {surface_flux:.2e} H/m2/s")

Surface flux = 4.72e+00 H/m2/s
